# Libraries and Global Variables

In [15]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
from IPython.display import display, Markdown

import arcpy

#import arcgis libraries
from arcgis.gis import *
gis = GIS()

In [16]:
# web app folder
folderWebAppDataFolder = r"..\widgets\Resiliency\data"

In [17]:
working_directory = os.getcwd()
data_folder         = os.path.join(working_directory, "data"        )
intermediate_folder = os.path.join(working_directory, "intermediate")
results_folder      = os.path.join(working_directory, "results"     )

print(data_folder)
print(intermediate_folder)
print(results_folder)

e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\data
e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\intermediate
e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\results


In [18]:
processGdb = os.path.join(intermediate_folder,'process.gdb')
processGdb

'e:\\WebAppBuilderForArcGIS_v2.20\\server\\apps\\15\\_dataprep\\intermediate\\process.gdb'

In [19]:
def delIfExists(obj):
    try:
        if arcpy.Exists(obj):
            arcpy.Delete_management(obj)
            #print ("DELTED: " + obj)
        #else:
            #print ("DOESN'T EXIST: " + obj)
    except:
        #print ("PASSED: " + obj)
        pass

# Download Data

In [20]:
# Read in RTP Projects
#item  = gis.content.get("d1bbd19922db433bb6309841ee4f7536")
#lyrRtpPoints = item.layers[0] # points layer is first layer
#lyrRtpLines  = item.layers[1] # lines layer is second layer

fcRtpLinesIn             = os.path.join(intermediate_folder,r'process.gdb\RTP_2023_Draft_lines')
fcRtpLinesSplitPointsTmp = os.path.join(intermediate_folder,r'process.gdb\RTP_2023_Draft_lines_splitpoints_temp')
fcRtpLinesSegsTmp        = os.path.join(intermediate_folder,r'process.gdb\RTP_2023_Draft_lines_segs_temp')
fcRtpLinesSegs           = os.path.join(intermediate_folder,r'process.gdb\RTP_2023_Draft_lines_segs')

split_length = .125 # miles


In [21]:
dfCategories = pd.DataFrame([
  ["HY", "Hydrological"                         ],
  ["HW", "Habitat and Wildlife"                 ],
  ["HM", "Hazardous Materials & Contamination"  ],
  ["CH", "Cultural and Historic Resources"      ],
  ["EJ", "Environmental Justice Consideration"  ],
  ["OS", "Open Space, Parks, and Recreation"    ],
  ["AF", "Agricultural and Farmland"            ],
  ["LU", "Land Use"                             ],
  ["FP", "Floodplains"                          ],
  ["SS", "Steep Slopes"                         ],
  ["GH", "Geological Hazards"                   ]
], columns=['CategoryCode','CategoryName'])
display(dfCategories)

# quick view layers: https://www.arcgis.com/apps/mapviewer/index.html?layers=74a6478eae424c17b96d1677ee1a4654

dfOpenDataLayers = pd.DataFrame([
  ["HW", "PAPR", "Public Access Properties (WMA)"       , "56aadb826ef3442db805553caf8126c0", "", 1.5],
  ["HW", "ELKS", "Elk"                                  , "224dbcbdca174d3e8648a8b5693fd130", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "DEER", "Mule Deer"                            , "a4ea235cef824862b560c693b4f7959b", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "PRNG", "Pronghorn"                            , "1cd6e0315a19473d925427f7cc1d02c1", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "TURK", "Turkey"                               , "9654e665cccd4884adb6a416df4d3444", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'" # no turkeys in crucial: "VALUE=='crucial'"
  ["HW", "MOOS", "Moose"                                , "e66c061ebd4346e3b16ed8786f5d5d78", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "BEAR", "Black Bear"                           , "6440f19a05c149b783baf1705e5f2a51", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "HARE", "Snowshoe Hare"                        , "d151c91e95c74b329814319f3c20b145", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "GOAT", "Mountain Goat"                        , "b6b369f865d243399bfc80908426d31b", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "DGRS", "Dusky Grouse"                         , "10382c7f5f384fc692d094fcb65bf251", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "RGRS", "Ruffed Grouse"                        , "a3d3fe1f13324882aa4b7f67253b1c06", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "HPAR", "Hungarian Partridge"                  , "7a7c3a873f0148ab8a188f743b7acff2", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "CHKR", "Chukar"                               , "97c6f7c74f704f84b687eb0746cc07ec", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "BISN", "Bison"                                , "9bed043eb87641db88424d1080d03045", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "BGHR", "Rocky Mountain Bighorn Sheep"         , "82b1e7ce1ec74920bcd40cd7e729c9b3", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "RNPH", "Ringed-Neck Pheasant"                 , "56acb32933f4463bb81cebb834595905", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "CSTG", "Columbian Sharp-Tailed Grouse"        , "c8e7ae2533d84a9a98268357dafa15da", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "CAQU", "California Quail"                     , "d5dae6ecfc8d490b87f1f67fd4d16fd9", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "CABS", "California Bighorn Sheep"             , "4c11e02a83f148dcafacbe203efc7432", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "BTPG", "Band-Tailed Pigeon"                   , "524a4c4b704244d4979b4d2b56f1d937", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "GMQU", "Gambel's Quail"                       , "95a538966055498e956817bad0a5835c", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "WTPT", "White-Tailed Ptarmigan"               , "170dbdc20fcc4a138022738167c4a765", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "WWDV", "White-Winged Dove"                    , "a2d03a434d6b4b189784434014fe0eef", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "UDBH", "Utah Desert Bighorn Sheep"            , "8c19124fbce245b5942c9e48c0568fab", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HW", "WTJR", "White-Tailed Jackrabbit"              , "1b02a508fa334d33ba18c0156d027751", "", 1.5], # "VALUE=='crucial' | VALUE=='NA'"
  ["HM", "BFWT", "Brownfields Targeted for Cleanup"     , "630351c0db444fe7933a08eab40e573e", "", 1.5],
  ["HM", "BFNT", "Brownfields Not Targeted for Cleanup" , "5fd65304423d40b38b4d93e50698db70", "", 1.5],
  ["HM", "HZWO", "Hazardous Waste & Used Oil Facilities", "8e9e6310622043bca68ffc2298959126", "", 1.5],
  ["HM", "LLWD", "Low-Level Waste & Uranium Mills"      , "2c81695cbc644afdba7687e0340b5763", "", 1.5],
  ["HM", "SWLN", "Solid Waste Landfills"                , "1ce0d598a0e440558c63535661dc2399", "", 1.5],
  ["HM", "FUDS", "Formerly Used Defense Sites"          , "dbcdc13c51614d109102f1251fe9530b", "", 1.5],
  ["HM", "MMRP", "Military Munitions Response Program"  , "c77c53794aeb4c448e1307b31c581909", "", 1.5],
  ["HM", "EWAS", "Enforceable Written Assurances"       , "7299aa0fdc80426ca0e42c42f59e1518", "", 1.5],
  ["HM", "NPRL", "National Priorities List"             , "f9c01f05050c40d7b86dade7dd877cf9", "", 1.5],
  ["HM", "SFST", "Superfunded Sites"                    , "929a2e7a8d5347b8b3248a450cc42e33", "", 1.5],
  ["HM", "ICBZ", "Institutional Control Buffer Zones"   , "d8ea2ca541724277b01726d98197da32", "", 1.5],
  ["HM", "UTRI", "Utah Toxic Release Inventory Tier II" , "2f725ff758e24dc496b17f87b7301ed8", "", 1.5],
  ["HM", "TXRI", "Toxic Release Inventory"              , "132ac94ba71d4e9b9bcffa57b07feb54", "", 1.5],
  ["HM", "VCUP", "Voluntary Cleanup Program"            , "e0316bc7899d4e4896e88fe3a46aafa8", "", 1.5],
  ["HM", "PSTF", "Petroleum Storage Tank Facilities"    , "4d31fef4543f449ebe04a84909a11023", "", 1.5],
  ["CH", "UHDS", "Utah Historic Districts"              , "eecae7fdf65f47aeae12ba817aa0d66b", "", 1.5],
  ["EJ", "EQFA", "Equity Focus Areas"                   , "2ed6d9b418d942b79323e6be7209d63c", "", 1.5],
  ["OS", "LCPK", "Local Parks"                          , "818536733987486b8082e4c5dec0a1dd", "", 1.5],
  ["OS", "LOWN", "Land Ownership"                       , "929fe39efa1d43ac8b151c2898ad3efe", "", 1.5],
  #["AF", "WRLN", "Water-Related Land"                   , "e3e7fc9316bb4ad09474401ff46e734f", "", 1.5],
  ["HY", "WETL", "Wetlands"                             , "dbc6a99d04944dab9f23bb6286843a1d", "", 1.5],
  ["HY", "RIPA", "Riparians"                            , "74a6478eae424c17b96d1677ee1a4654", "", 1.5]
],columns=['CategoryCode','LayerCode','LayerName','OpenDataKey','QueryFilter','SearchBuffer'])
display(dfOpenDataLayers)

# example: https://opendata.arcgis.com/datasets/a37e501176dd400c9c7498c29e766ba2_0.gdb

#item  = gis.content.get("630351c0db444fe7933a08eab40e573e")
#layer = item.layers[0]

#dfDownloadedLayers = pd.DataFrame([
#  [],
#  [],
#],columns=['LayerName','Filename'])

,CategoryCode,CategoryName
0,HY,Hydrological
1,HW,Habitat and Wildlife
2,HM,Hazardous Materials & Contamination
3,CH,Cultural and Historic Resources
4,EJ,Environmental Justice Consideration
5,OS,"Open Space, Parks, and Recreation"
6,AF,Agricultural and Farmland
7,LU,Land Use
8,FP,Floodplains
9,SS,Steep Slopes


,CategoryCode,LayerCode,LayerName,OpenDataKey,QueryFilter,SearchBuffer
0,HW,PAPR,Public Access Properties (WMA),56aadb826ef3442db805553caf8126c0,,1.5
1,HW,ELKS,Elk,224dbcbdca174d3e8648a8b5693fd130,,1.5
2,HW,DEER,Mule Deer,a4ea235cef824862b560c693b4f7959b,,1.5
3,HW,PRNG,Pronghorn,1cd6e0315a19473d925427f7cc1d02c1,,1.5
4,HW,TURK,Turkey,9654e665cccd4884adb6a416df4d3444,,1.5
5,HW,MOOS,Moose,e66c061ebd4346e3b16ed8786f5d5d78,,1.5
6,HW,BEAR,Black Bear,6440f19a05c149b783baf1705e5f2a51,,1.5
7,HW,HARE,Snowshoe Hare,d151c91e95c74b329814319f3c20b145,,1.5
8,HW,GOAT,Mountain Goat,b6b369f865d243399bfc80908426d31b,,1.5
9,HW,DGRS,Dusky Grouse,10382c7f5f384fc692d094fcb65bf251,,1.5


In [22]:
#Check for duplicate codes:
dfOpenDataLayers['LayerCode'].duplicated().any()

False

In [23]:
#item1  = gis.content.get("a4ea235cef824862b560c693b4f7959b")
#item2  = gis.content.get("dbc6a99d04944dab9f23bb6286843a1d")
#flayer1 = item1.layers[0]
#flayer2 = item2.layers[0]
#sdfTemp1 = pd.DataFrame.spatial.from_layer(flayer1)
#sdfTemp2 = pd.DataFrame.spatial.from_layer(flayer1)
#
#sdfTemp1.query("VALUE=='crucial'")


# Split RTP polyline projects into predefined lengths

In [24]:
##import arcgis libraries
#from arcgis.gis import *
#gis = GIS()
#
##create map1 that centers on Salt Lake (can replace 'Salt Lake' with any place name or leave blank for entire world)
#map1 = gis.map('Salt Lake')
#
#sdf = pd.DataFrame.spatial.from_featureclass(fcRtpLinesIn)
#
##plot sdf using class breaks and blue colors for 'columnname' field
##layer will be added to 'map1' above
#sdf.spatial.plot(map_widget = map1)
#
##show map1
#map1

In [25]:
from IPython.display import clear_output, display

bSplitSegs = False

if bSplitSegs:

    delIfExists(fcRtpLinesSegs)

    bFirst = True

    # Create a search cursor for the feature class
    fields = ["OID@", "SHAPE@", "gis_id"]
    cursor = arcpy.da.SearchCursor(fcRtpLinesIn, fields)

    # Get the total number of rows in the feature class
    count = arcpy.GetCount_management(fcRtpLinesIn)[0]

    # need to loop through each polyline separately since there are overlapping projects and have to be treated separately
    # Loop through each polyline in the input feature class
    for i, row in enumerate(cursor):

        # Get the OID and shape of the current polyline
        oid      = row[0]
        polyline = row[1]
        gid      = row[2]
        
        # delete temp layers
        delIfExists(fcRtpLinesSplitPointsTmp)
        delIfExists(fcRtpLinesSegsTmp)


        # Generate points along the polyline at the specified interval
        arcpy.GeneratePointsAlongLines_management(polyline, fcRtpLinesSplitPointsTmp, 'DISTANCE', Distance = str(split_length) + ' miles')
        
        # Split the polyline at the generated points
        arcpy.SplitLineAtPoint_management(polyline, fcRtpLinesSplitPointsTmp, fcRtpLinesSegsTmp, '5 feet')

        
        # Add a new field to the output feature class to store the original OID of the polyline
        arcpy.AddField_management(fcRtpLinesSegsTmp, "gis_id", "TEXT")

        # Populate the ORIG_OID field with the OID of the original polyline
        with arcpy.da.UpdateCursor(fcRtpLinesSegsTmp, ["gis_id"]) as update_cursor:
            for update_row in update_cursor:
                update_row[0] = gid
                update_cursor.updateRow(update_row)

        if bFirst:
            arcpy.management.Copy(fcRtpLinesSegsTmp, fcRtpLinesSegs)
            bFirst = False
        else:
            arcpy.management.Append(fcRtpLinesSegsTmp, fcRtpLinesSegs)

        # Update the progressor
        progress = "Processing polylines. Done processing: " + str(i+1) + " of " + str(count)
        display(progress, display_id="progress")

        # Clear the previous progress message
        clear_output(wait=True)

    # Clean up the cursor and progressor
    del cursor

took 49 minutes to run

In [26]:
# delete temp layers
delIfExists(fcRtpLinesSplitPointsTmp)
delIfExists(fcRtpLinesSegsTmp)

In [27]:
##import arcgis libraries
#from arcgis.gis import *
#gis = GIS()
#
##create map1 that centers on Salt Lake (can replace 'Salt Lake' with any place name or leave blank for entire world)
#map2 = gis.map('Salt Lake')
#
#sdfSegs = pd.DataFrame.spatial.from_featureclass(fcRtpLinesSegs)
#
##plot sdf using class breaks and blue colors for 'columnname' field
##layer will be added to 'map1' above
#sdfSegs.spatial.plot(map_widget = map2)
#
##show map1
#map2

In [28]:
bProcess         = False
bReDownloadAll   = False
bReJoinAll       = False

gis = GIS(expiration=9999)

if bProcess: 

    import time

    start_time = time.time()

    for index, row in dfOpenDataLayers.iterrows():

        # display all the variables used
        print (row['LayerCode'] + ' -  ' + row['OpenDataKey'] + ' - ' + str(row['SearchBuffer']) + ' -  ' + row['QueryFilter'])

        #try:
        _fc_download = os.path.join(intermediate_folder,'process.gdb\\' + row['LayerCode'] + '_orig_fltr')
        _fc_join     = os.path.join(intermediate_folder,'process.gdb\\' + row['LayerCode'] + '_joined'   )

        # check to see if replace all toggle is on OR if the feature class already exists in the gdb
        if bReDownloadAll or not arcpy.Exists(_fc_download):

            # delete feature class if it exists
            delIfExists(_fc_download)

            # get data from web
            print ("downloading...")
            _item  = gis.content.get(row['OpenDataKey'])
            _flayer = _item.layers[0]
            _sdfLyr = pd.DataFrame.spatial.from_layer(_flayer)

            # filter data based on query
            print ("filtering...")
            if bool(row['QueryFilter']):
                _sdfLyr = _sdfLyr.query(row['QueryFilter'])

            # if sdf is not empty
            if not _sdfLyr.empty:
                print ("exporting intermediate layer...")
                _sdfLyr.spatial.to_featureclass(_fc_download)

        if bReJoinAll or not arcpy.Exists(_fc_join):

            # only join again if download has been created
            if arcpy.Exists(_fc_download):

                # delete feature class if it exists
                delIfExists(_fc_join)

                # get closest distance
                print ("joining...")
                arcpy.analysis.SpatialJoin(fcRtpLinesSegs, _fc_download, _fc_join, "JOIN_ONE_TO_ONE", "KEEP_COMMON", match_option="CLOSEST", search_radius=str(row['SearchBuffer']) + " mile", distance_field_name="meters" + row['LayerCode'])
                del _sdfLyr
            else:
                print('no features to join to')
                
        end_time = time.time()

        hours, rem = divmod(end_time - start_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"done... elapsed time: {int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds") 
        #except:
        #    print ("ERROR: " + row['LayerCode'])

print ('ALL DONE!')

ALL DONE!


In [29]:
##import arcgis libraries
#from arcgis.gis import *
#gis = GIS()
#
##create map1 that centers on Salt Lake (can replace 'Salt Lake' with any place name or leave blank for entire world)
#map3 = gis.map('Salt Lake')
#
#sdf = pd.DataFrame.spatial.from_featureclass(os.path.join(intermediate_folder,'process.gdb\\WETL_joined'))
#
##plot sdf using class breaks and blue colors for 'columnname' field
##layer will be added to 'map1' above
#sdf.spatial.plot(map_widget = map3)
#
##show map1
#map3

In [30]:
dfD = pd.DataFrame()

for index, row in dfOpenDataLayers.iterrows():
    #try:
    print(row['LayerCode'])
    _fc = os.path.join(processGdb,row['LayerCode'] + '_joined')
    if arcpy.Exists(_fc):
        #display(_fc)

        _sdf = pd.DataFrame.spatial.from_featureclass(_fc)
        
        df = _sdf[['gis_id',"meters" + row['LayerCode'],'ORIG_SEQ']].copy()
        df['LayerCode'] = row['LayerCode']

        df.columns=(['gis_id','dist_meters','ORIG_SEQ','LayerCode'])

        #display(_sdf[['gis_id',"meters" + row['LayerCode']]])

        dfD = pd.concat([dfD,df])

dfD['dist_feet'] = dfD['dist_meters'] / 3.28084

dfD = dfD.drop(columns=('dist_meters'))

dfD

PAPR
ELKS
DEER
PRNG
TURK
MOOS
BEAR
HARE
GOAT
DGRS
RGRS
HPAR
CHKR
BISN
BGHR
RNPH
CSTG
CAQU
CABS
BTPG
GMQU
WTPT
WWDV
UDBH
WTJR
BFWT
BFNT
HZWO
LLWD
SWLN
FUDS
MMRP
EWAS
NPRL
SFST
ICBZ
UTRI
TXRI
VCUP
PSTF
UHDS
EQFA
LCPK
LOWN
WETL
RIPA


,gis_id,ORIG_SEQ,LayerCode,dist_feet
0,A-B-4,1,PAPR,616.443915
1,A-B-4,2,PAPR,596.895954
2,A-B-4,3,PAPR,587.911962
3,A-B-4,4,PAPR,587.796539
4,A-B-4,5,PAPR,615.124685
...,...,...,...,...
4464,729,21,RIPA,508.894502
4465,729,19,RIPA,624.271305
4466,729,20,RIPA,563.818440
4467,729,22,RIPA,507.683681


In [31]:
# DON'T USE NONE OR NEW
dfDistance = dfD[['gis_id','dist_feet','ORIG_SEQ','LayerCode']].copy()
dfDistance.columns =['g','ft','s','l']
dfDistance['id'] = dfDistance['g'].astype(str) + '_' + dfDistance['s'].astype(str)
dfDistance = dfDistance[['l','id','ft']]
dfDistance = dfDistance.groupby(['l','id'],as_index=False).agg(ft=('ft','mean'))
dfDistance['ft'] = dfDistance['ft'].astype(int)
dfDistance

,l,id,ft
0,BEAR,11_64,694
1,BEAR,11_65,633
2,BEAR,11_66,572
3,BEAR,11_67,527
4,BEAR,128_18,682
...,...,...,...
346854,WETL,WestWeber_87,0
346855,WETL,WestWeber_88,0
346856,WETL,WestWeber_89,0
346857,WETL,WestWeber_9,89


In [32]:
dfDistance_pivot = dfD.pivot_table(index=['gis_id','ORIG_SEQ'],columns='LayerCode',aggfunc='mean')
dfDistance_pivot.fillna(-1,inplace=True)
dfDistance_pivot = dfDistance_pivot.astype(int)
dfDistance_pivot.columns = dfDistance_pivot.columns.droplevel(0)
dfDistance_pivot.reset_index(inplace=True)
dfDistance_pivot

LayerCode,gis_id,ORIG_SEQ,BEAR,BFNT,BFWT,BTPG,CAQU,CHKR,DEER,DGRS,...,RNPH,SFST,SWLN,TURK,TXRI,UHDS,UTRI,VCUP,WETL,WTJR
0,10,1,-1,311,-1,-1,0,-1,-1,-1,...,0,304,-1,-1,-1,-1,111,-1,53,-1
1,10,2,-1,311,-1,-1,0,-1,-1,-1,...,0,302,-1,-1,-1,-1,151,-1,50,-1
2,10,3,-1,317,-1,-1,0,-1,-1,-1,...,0,303,-1,-1,-1,-1,201,-1,57,-1
3,10,4,-1,335,-1,-1,0,-1,-1,-1,...,0,315,-1,-1,-1,-1,240,-1,90,-1
4,10,5,-1,371,-1,-1,0,-1,-1,-1,...,0,282,-1,-1,-1,-1,215,-1,101,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25339,WestWeber,85,-1,-1,-1,-1,-1,-1,-1,-1,...,0,-1,-1,-1,-1,-1,-1,-1,2,-1
25340,WestWeber,86,-1,-1,-1,-1,-1,-1,-1,-1,...,0,-1,-1,-1,-1,-1,-1,-1,0,-1
25341,WestWeber,87,-1,-1,-1,-1,-1,-1,-1,-1,...,0,-1,-1,-1,-1,-1,-1,-1,0,-1
25342,WestWeber,88,-1,-1,-1,-1,-1,-1,-1,-1,...,0,722,-1,-1,-1,-1,-1,-1,0,-1


# Web App Data

In [33]:
sdfRtpLinesSegs = pd.DataFrame.spatial.from_featureclass(fcRtpLinesSegs)
sdfRtpLinesSegs = sdfRtpLinesSegs.groupby(['gis_id','ORIG_SEQ'],as_index=False).agg(SHAPE=('SHAPE','first'))

#display(sdfRtpLinesSegs)

sdfRtpLinesSegsWithDist = pd.DataFrame.merge(sdfRtpLinesSegs,dfDistance_pivot,on=('gis_id','ORIG_SEQ'),how='left')

# iterate through the list of columns and add them to the DataFrame
for col in dfOpenDataLayers['LayerCode'].unique():
    if col not in sdfRtpLinesSegsWithDist.columns:
        sdfRtpLinesSegsWithDist[col] = None
        
sdfRtpLinesSegsWithDist.fillna(-1,inplace=True)

display(sdfRtpLinesSegsWithDist)

,gis_id,ORIG_SEQ,SHAPE,BEAR,BFNT,BFWT,BTPG,CAQU,CHKR,DEER,...,PRNG,BISN,BGHR,CSTG,CABS,GMQU,WTPT,WWDV,UDBH,LLWD
0,10,1,"{'paths': [[[413426.50009999983, 4507190.0601]...",-1,311,-1,-1,0,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,10,2,"{'paths': [[[413225.3596999999, 4507186.7063],...",-1,311,-1,-1,0,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,10,3,"{'paths': [[[413024.19849999994, 4507185.1809]...",-1,317,-1,-1,0,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,10,4,"{'paths': [[[412823.03369999956, 4507183.9804]...",-1,335,-1,-1,0,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,10,5,"{'paths': [[[412626.7885999996, 4507150.073899...",-1,371,-1,-1,0,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25339,WestWeber,85,"{'paths': [[[401672.7982000001, 4566569.049000...",-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
25340,WestWeber,86,"{'paths': [[[401471.69130000006, 4566573.3893]...",-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
25341,WestWeber,87,"{'paths': [[[401285.83779999986, 4566643.4121]...",-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
25342,WestWeber,88,"{'paths': [[[401164.80399999954, 4566800.8816]...",-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [53]:
# export segs
bExportSegs = False

if bExportSegs:
    sdfRtpLinesSegsWithDist.spatial.to_featureclass(os.path.join(results_folder,"resiliency_segs.shp"), sanitize_columns=False)
    print("SEGMENTS EXPORTED")
    sdfRtpLinesIn = pd.DataFrame.spatial.from_featureclass(fcRtpLinesIn)[['GIS_ID','SHAPE']]
    sdfRtpLinesIn.spatial.to_featureclass(os.path.join(results_folder,"resiliency_projects_lines.shp"))
    print("PROJECTS EXPORTED")
else:
    print("SEGMENTS NOT EXPORTED")
    print("PROJECTS NOT EXPORTED")


SEGMENTS EXPORTED
PROJECTS EXPORTED


In [35]:
sdfRtpLinesSegs

,gis_id,ORIG_SEQ,SHAPE
0,10,1,"{""paths"": [[[413426.50009999983, 4507190.0601]..."
1,10,2,"{""paths"": [[[413225.3596999999, 4507186.7063],..."
2,10,3,"{""paths"": [[[413024.19849999994, 4507185.1809]..."
3,10,4,"{""paths"": [[[412823.03369999956, 4507183.9804]..."
4,10,5,"{""paths"": [[[412626.7885999996, 4507150.073899..."
...,...,...,...
25339,WestWeber,85,"{""paths"": [[[401672.7982000001, 4566569.049000..."
25340,WestWeber,86,"{""paths"": [[[401471.69130000006, 4566573.3893]..."
25341,WestWeber,87,"{""paths"": [[[401285.83779999986, 4566643.4121]..."
25342,WestWeber,88,"{""paths"": [[[401164.80399999954, 4566800.8816]..."


In [36]:
dfDistance_pivot[['gis_id']].drop_duplicates()

LayerCode,gis_id
0,10
17,101
46,103
68,106
82,107
...,...
24938,StGrid
25019,StGrid_2
25125,StGrid_2_BRT
25242,TRAXImp


In [37]:
# export jsons for categories and layers
dfCategories    .to_json(os.path.join(folderWebAppDataFolder,'cats.json'),orient="table")
dfOpenDataLayers.to_json(os.path.join(folderWebAppDataFolder,'lyrs.json'),orient="table")


# get all gisids
sdfRtpLinesSegs[['gis_id']].drop_duplicates().rename(columns={'gis_id':'g'}).to_json(os.path.join(folderWebAppDataFolder,'gids.json'),index=False,orient="table")

# get only gisid and seq of 
dfDistance_pivot[['gis_id','ORIG_SEQ']].drop_duplicates().rename(columns={'gis_id':'g','ORIG_SEQ':'s'}).to_json(os.path.join(folderWebAppDataFolder,'segs.json'),index=False,orient="table")

In [38]:
dfDistance.dtypes

l     object
id    object
ft     int32
dtype: object

In [39]:
# REDO

# FILE BY BUFFER - GIS_ID - CATEGORY
# ROW BY ORIG_SEQ
# RECORDS: 

lstBuffers = [100,200,300,400,500,600]

dfBuffers = dfDistance.copy()

for b in lstBuffers:
    print(b)
    dfBuffers.loc[(dfBuffers['ft']<b),'b' + str(b)] = 1
    dfBuffers['b' + str(b)] = dfBuffers['b' + str(b)].fillna(0)
    dfBuffers['b' + str(b)] = dfBuffers['b' + str(b)].astype(int)


display(dfBuffers)
dfBuffers.drop(columns=('ft'),inplace=True)

display(dfBuffers)
# export one dist file per gis id

#for index, row in dfDistance[['l']].drop_duplicates().iterrows():
#    print(row['l'])
#    _dfExport = dfDistance[dfDistance['l']==row['l']][['id','d']].set_index('id').to_json(os.path.join(folderWebAppDataFolder + '\\dist' , row['l'] + '.json'),orient='index')

100
200
300
400
500
600


,l,id,ft,b100,b200,b300,b400,b500,b600
0,BEAR,11_64,694,0,0,0,0,0,0
1,BEAR,11_65,633,0,0,0,0,0,0
2,BEAR,11_66,572,0,0,0,0,0,1
3,BEAR,11_67,527,0,0,0,0,0,1
4,BEAR,128_18,682,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
346854,WETL,WestWeber_87,0,1,1,1,1,1,1
346855,WETL,WestWeber_88,0,1,1,1,1,1,1
346856,WETL,WestWeber_89,0,1,1,1,1,1,1
346857,WETL,WestWeber_9,89,1,1,1,1,1,1


,l,id,b100,b200,b300,b400,b500,b600
0,BEAR,11_64,0,0,0,0,0,0
1,BEAR,11_65,0,0,0,0,0,0
2,BEAR,11_66,0,0,0,0,0,1
3,BEAR,11_67,0,0,0,0,0,1
4,BEAR,128_18,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
346854,WETL,WestWeber_87,1,1,1,1,1,1
346855,WETL,WestWeber_88,1,1,1,1,1,1
346856,WETL,WestWeber_89,1,1,1,1,1,1
346857,WETL,WestWeber_9,1,1,1,1,1,1


In [40]:
dfBuffersWithCat = pd.DataFrame.merge(dfBuffers, dfOpenDataLayers, left_on='l', right_on='LayerCode')
#display(dfBuffersWithCat)

for b in lstBuffers:
    print(b)
    for c in dfCategories['CategoryCode'].unique():
        print(c)
        _lyrs = (dfOpenDataLayers[dfOpenDataLayers['CategoryCode']==c]['LayerCode'].unique())
        _df = dfBuffersWithCat[(dfBuffersWithCat['CategoryCode']==c)][['id','l','CategoryCode','b' + str(b)]]
        _df = _df[_df['b' + str(b)]>0]
        _df.loc[(_df['b' + str(b)]==1),'value'] = _df['l']
        #display(_df)

        _dfp = _df.pivot(index='id', columns='l', values='value')
        _dfp.reset_index(inplace=True)
        _dfp.fillna('',inplace=True)
        _dfp[c] = _dfp[_df['l'].unique()].apply(lambda x: ",".join(filter(None, x)), axis =1)
        _dfp = _dfp[['id',c]]
        _dfp.set_index('id',inplace=True)
        #display(_dfp)

        _dfp.to_json(os.path.join(folderWebAppDataFolder + '\\within_buffers' , 'b' + str(b) + '_' + c + '.json'),orient='index')

100
HY
HW
HM
CH
EJ
OS
AF
LU
FP
SS
GH
200
HY
HW
HM
CH
EJ
OS
AF
LU
FP
SS
GH
300
HY
HW
HM
CH
EJ
OS
AF
LU
FP
SS
GH
400
HY
HW
HM
CH
EJ
OS
AF
LU
FP
SS
GH
500
HY
HW
HM
CH
EJ
OS
AF
LU
FP
SS
GH
600
HY
HW
HM
CH
EJ
OS
AF
LU
FP
SS
GH
